In [3]:
import yfinance as yf
import pandas as pd
import pandas_ta as ta
from datetime import datetime
import logging

# Clear the log file before starting
with open('supertrend_analysis.log', 'w'):
    pass

# Configure logging
logging.basicConfig(level=logging.INFO, 
                    format='%(asctime)s - %(levelname)s - %(message)s')

# Add file handler
file_handler = logging.FileHandler('supertrend_analysis.log')
file_handler.setLevel(logging.INFO)
file_handler.setFormatter(logging.Formatter('%(asctime)s - %(levelname)s - %(message)s'))

# Add console handler
console_handler = logging.StreamHandler()
console_handler.setLevel(logging.INFO)
console_handler.setFormatter(logging.Formatter('%(asctime)s - %(levelname)s - %(message)s'))

# Get the root logger
logger = logging.getLogger()
logger.addHandler(file_handler)
logger.addHandler(console_handler)

def analyze_supertrend(ticker):
    """Analyze a single stock using Supertrend indicator"""
    try:
        # Download stock data
        stock = yf.Ticker(ticker + ".NS")
        df = stock.history(interval='1wk', period='5y')  # Get maximum available history
        
        # Calculate Supertrend
        supertrend = df.ta.supertrend(length=9, multiplier=2)
        
        # Merge Supertrend results with the original dataframe
        df = pd.concat([df, supertrend], axis=1)
        
        # Log last 4 weeks of data
        logging.info(f"\n=== Last 4 Weeks Data for {ticker} ===")
        logging.info("=" * 80)
        recent_data = df.tail(4)
        for idx, row in recent_data.iterrows():
            date = idx.strftime('%Y-%m-%d')
            price = row['Close']
            supertrend_value = row['SUPERT_9_2.0']
            trend = "UPTREND" if row['SUPERTd_9_2.0'] == 1 else "DOWNTREND"
            distance = abs(price - supertrend_value)
            distance_percent = (distance / price) * 100
            
            logging.info(f"Date: {date}")
            logging.info(f"Close Price: {price:.2f}")
            logging.info(f"Supertrend: {supertrend_value:.2f}")
            logging.info(f"Trend: {trend}")
            logging.info(f"Distance to Supertrend: {distance:.2f} ({distance_percent:.2f}%)")
            logging.info("-" * 50)
        
        # Get last week's data for Excel export
        last_week = df.iloc[-1]
        current_price = last_week['Close']
        supertrend_value = last_week['SUPERT_9_2.0']
        trend_direction = last_week['SUPERTd_9_2.0']
        
        # Make trading decision
        if trend_direction == 1:
            decision = "BUY"
            reason = "Stock is in UPTREND"
        elif trend_direction == -1:
            decision = "SELL"
            reason = "Stock is in DOWNTREND"
        else:
            decision = "NO SIGNAL"
            reason = "No clear trend"
            
        # Calculate distance
        distance = abs(current_price - supertrend_value)
        distance_percent = (distance / current_price) * 100
        
        return {
            'Ticker': ticker,
            'Date': last_week.name.strftime('%Y-%m-%d'),
            'Current Price': round(current_price, 2),
            'Supertrend Value': round(supertrend_value, 2),
            'Decision': decision,
            'Reason': reason,
            'Distance to Supertrend': round(distance, 2),
            'Distance Percentage': round(distance_percent, 2)
        }
        
    except Exception as e:
        logging.error(f"Error analyzing {ticker}: {str(e)}")
        return {
            'Ticker': ticker,
            'Date': None,
            'Current Price': None,
            'Supertrend Value': None,
            'Decision': 'ERROR',
            'Reason': f'Error: {str(e)}',
            'Distance to Supertrend': None,
            'Distance Percentage': None
        }

# Read tickers from CSV file
try:
    tickers_df = pd.read_csv('tickers.csv')
    tickers = tickers_df['Ticker'].tolist()
except Exception as e:
    logging.error(f"Error reading CSV file: {str(e)}")
    logging.error("Please ensure you have a 'tickers.csv' file with a 'Ticker' column")
    exit()

# Analyze all stocks
results = []
for ticker in tickers:
    logging.info(f"\nAnalyzing {ticker}...")
    result = analyze_supertrend(ticker)
    results.append(result)

# Create results DataFrame
results_df = pd.DataFrame(results)

# Add timestamp to filename
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
output_filename = f'supertrend_analysis_{timestamp}.xlsx'

# Save to Excel
results_df.to_excel(output_filename, sheet_name='Supertrend Analysis', index=False)

logging.info(f"\nAnalysis complete! Results saved to '{output_filename}'")

2025-03-21 12:55:32,484 - INFO - 
Analyzing MAZDOCK...
2025-03-21 12:55:32,484 - INFO - 
Analyzing MAZDOCK...
2025-03-21 12:55:32,484 - INFO - 
Analyzing MAZDOCK...
2025-03-21 12:55:32,820 - INFO - 
=== Last 4 Weeks Data for MAZDOCK ===
2025-03-21 12:55:32,820 - INFO - 
=== Last 4 Weeks Data for MAZDOCK ===
2025-03-21 12:55:32,820 - INFO - 
=== Last 4 Weeks Data for MAZDOCK ===
2025-03-21 12:55:32,821 - INFO - ================================================================================
2025-03-21 12:55:32,821 - INFO - ================================================================================
2025-03-21 12:55:32,821 - INFO - ================================================================================
2025-03-21 12:55:32,822 - INFO - Date: 2025-02-24
2025-03-21 12:55:32,822 - INFO - Date: 2025-02-24
2025-03-21 12:55:32,822 - INFO - Date: 2025-02-24
2025-03-21 12:55:32,823 - INFO - Close Price: 2131.05
2025-03-21 12:55:32,823 - INFO - Close Price: 2131.05
2025-03-21 12:55:32

In [2]:
#With logging file


import yfinance as yf
import pandas as pd
import pandas_ta as ta
from datetime import datetime
import logging
import psycopg2

# Clear the log file before starting
with open('supertrend_analysis.log', 'w'):
    pass

# Configure logging
logging.basicConfig(level=logging.INFO, 
                    format='%(asctime)s - %(levelname)s - %(message)s')

# Add file handler
file_handler = logging.FileHandler('supertrend_analysis.log')
file_handler.setLevel(logging.INFO)
file_handler.setFormatter(logging.Formatter('%(asctime)s - %(levelname)s - %(message)s'))

# Add console handler
console_handler = logging.StreamHandler()
console_handler.setLevel(logging.INFO)
console_handler.setFormatter(logging.Formatter('%(asctime)s - %(levelname)s - %(message)s'))

# Get the root logger
logger = logging.getLogger()
logger.addHandler(file_handler)
logger.addHandler(console_handler)

def analyze_supertrend(ticker):
    """Analyze a single stock using Supertrend indicator"""
    try:
        # Download stock data
        stock = yf.Ticker(ticker + ".NS")
        df = stock.history(interval='1wk', period='5y')  # Get maximum available history
        
        # Calculate Supertrend
        supertrend = df.ta.supertrend(length=9, multiplier=2)
        
        # Merge Supertrend results with the original dataframe
        df = pd.concat([df, supertrend], axis=1)
        
        # Log last 4 weeks of data
        logging.info(f"\n=== Last 4 Weeks Data for {ticker} ===")
        logging.info("=" * 80)
        recent_data = df.tail(4)
        for idx, row in recent_data.iterrows():
            date = idx.strftime('%Y-%m-%d')
            price = row['Close']
            supertrend_value = row['SUPERT_9_2.0']
            trend = "UPTREND" if row['SUPERTd_9_2.0'] == 1 else "DOWNTREND"
            distance = abs(price - supertrend_value)
            distance_percent = (distance / price) * 100
            
            logging.info(f"Date: {date}")
            logging.info(f"Close Price: {price:.2f}")
            logging.info(f"Supertrend: {supertrend_value:.2f}")
            logging.info(f"Trend: {trend}")
            logging.info(f"Distance to Supertrend: {distance:.2f} ({distance_percent:.2f}%)")
            logging.info("-" * 50)
        
        # Get last week's data for Excel export
        last_week = df.iloc[-1]
        current_price = last_week['Close']
        supertrend_value = last_week['SUPERT_9_2.0']
        trend_direction = last_week['SUPERTd_9_2.0']
        
        # Make trading decision
        if trend_direction == 1:
            decision = "BUY"
            reason = "Stock is in UPTREND"
        elif trend_direction == -1:
            decision = "SELL"
            reason = "Stock is in DOWNTREND"
        else:
            decision = "NO SIGNAL"
            reason = "No clear trend"
            
        # Calculate distance
        distance = abs(current_price - supertrend_value)
        distance_percent = (distance / current_price) * 100
        
        return {
            'Ticker': ticker,
            'Date': last_week.name.strftime('%Y-%m-%d'),
            'Current Price': round(current_price, 2),
            'Supertrend Value': round(supertrend_value, 2),
            'Decision': decision,
            'Reason': reason,
            'Distance to Supertrend': round(distance, 2),
            'Distance Percentage': round(distance_percent, 2)
        }
        
    except Exception as e:
        logging.error(f"Error analyzing {ticker}: {str(e)}")
        return {
            'Ticker': ticker,
            'Date': None,
            'Current Price': None,
            'Supertrend Value': None,
            'Decision': 'ERROR',
            'Reason': f'Error: {str(e)}',
            'Distance to Supertrend': None,
            'Distance Percentage': None
        }

# Read tickers from CSV file
try:
    tickers_df = pd.read_csv('tickers.csv')
    tickers = tickers_df['Ticker'].tolist()
except Exception as e:
    logging.error(f"Error reading CSV file: {str(e)}")
    logging.error("Please ensure you have a 'tickers.csv' file with a 'Ticker' column")
    exit()

# Analyze all stocks
results = []
for ticker in tickers:
    logging.info(f"\nAnalyzing {ticker}...")
    result = analyze_supertrend(ticker)
    results.append(result)

# Create results DataFrame
results_df = pd.DataFrame(results)

#create a postgres database connection and insert the data check if table exists if not then create it also insert flag if flag value true then insert data else not
# Database connection parameters
db_params = {
    "dbname": "factor_investing",
        "user": "tushardesarda",
        "password": "",
        "host": "localhost",
        "port": "5432"
}

# Flag to control data insertion
insert_flag = True

if insert_flag:
    try:
        # Connect to the PostgreSQL database
        conn = psycopg2.connect(**db_params)
        cursor = conn.cursor()
        
        # Check if table exists, if not create it
        create_table_query = """
        CREATE TABLE IF NOT EXISTS supertrend_analysis (
            Ticker VARCHAR(10),
            Date DATE,
            Current_Price FLOAT,
            Supertrend_Value FLOAT,
            Decision VARCHAR(10),
            Reason TEXT,
            Distance_to_Supertrend FLOAT,
            Distance_Percentage FLOAT
        )
        """
        cursor.execute(create_table_query)
        conn.commit()
        
        # Insert data into the table
        insert_query = """
        INSERT INTO supertrend_analysis (Ticker, Date, Current_Price, Supertrend_Value, Decision, Reason, Distance_to_Supertrend, Distance_Percentage)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
        """
        for index, row in results_df.iterrows():
            cursor.execute(insert_query, (
                row['Ticker'], row['Date'], row['Current Price'], row['Supertrend Value'], 
                row['Decision'], row['Reason'], row['Distance to Supertrend'], row['Distance Percentage']
            ))
        conn.commit()
        
        logging.info("Data successfully inserted into the PostgreSQL database.")
        
    except Exception as e:
        logging.error(f"Error inserting data into PostgreSQL database: {str(e)}")
        
    finally:
        cursor.close()
        conn.close()
# Add timestamp to filename
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
output_filename = f'supertrend_analysis_{timestamp}.xlsx'

# Save to Excel
results_df.to_excel(output_filename, sheet_name='Supertrend Analysis', index=False)

logging.info(f"\nAnalysis complete! Results saved to '{output_filename}'")

2025-03-21 12:53:19,945 - INFO - 
Analyzing MAZDOCK...
2025-03-21 12:53:19,945 - INFO - 
Analyzing MAZDOCK...
2025-03-21 12:53:21,409 - INFO - 
=== Last 4 Weeks Data for MAZDOCK ===
2025-03-21 12:53:21,409 - INFO - 
=== Last 4 Weeks Data for MAZDOCK ===
2025-03-21 12:53:21,410 - INFO - ================================================================================
2025-03-21 12:53:21,410 - INFO - ================================================================================
2025-03-21 12:53:21,411 - INFO - Date: 2025-02-24
2025-03-21 12:53:21,411 - INFO - Date: 2025-02-24
2025-03-21 12:53:21,411 - INFO - Close Price: 2131.05
2025-03-21 12:53:21,411 - INFO - Close Price: 2131.05
2025-03-21 12:53:21,412 - INFO - Supertrend: 2472.97
2025-03-21 12:53:21,412 - INFO - Supertrend: 2472.97
2025-03-21 12:53:21,412 - INFO - Trend: DOWNTREND
2025-03-21 12:53:21,412 - INFO - Trend: DOWNTREND
2025-03-21 12:53:21,413 - INFO - Distance to Supertrend: 341.92 (16.04%)
2025-03-21 12:53:21,413 - INFO 

NameError: name 'cursor' is not defined

In [1]:
# -*- coding: utf-8 -*-
"""
Created on Fri Mar 21 12:24:19 2025

@author: admin
"""

import pandas as pd
import yfinance as yf
from datetime import datetime, timedelta
import pytz  # Import timezone library

def fetch_stock_data(ticker):
    """Fetch 1-year stock data from Yahoo Finance and store it in memory."""
    try:
        ticker_symbol = ticker + ".NS"
        print(f"Fetching data for {ticker_symbol} with daily interval for 1 year")
        
        stock = yf.Ticker(ticker_symbol)
        stock_data = stock.history(interval='1d', period='1y')  # Fetch daily data for 1 year
        
        if stock_data is None or stock_data.empty or 'Close' not in stock_data:
            print(f"No data available for {ticker_symbol}")
            return None
        
        # Ensure index is timezone-aware (in UTC)
        stock_data.index = stock_data.index.tz_convert('UTC')
        
        return stock_data['Close']
    except Exception as e:
        print(f"Error fetching data for {ticker}: {e}")
        return None

def get_nearest_price(stock_data, target_date):
    """Find the nearest available price for a given date."""
    if stock_data is None or stock_data.empty:
        return None

    # Ensure target_date is also timezone-aware (convert to UTC)
    target_date = pd.Timestamp(target_date).tz_localize('UTC')

    if target_date in stock_data.index:
        return stock_data.loc[target_date]
    else:
        return stock_data.asof(target_date)

def calculate_returns(ticker, stock_data):
    try:
        print(stock_data)
        if stock_data is None:
            print(f"Skipping return calculation for {ticker} due to missing data.")
            return [ticker, None, None, None]
        
        # Convert today and past dates to UTC
        today = datetime.today().replace(tzinfo=pytz.UTC).strftime('%Y-%m-%d')
        three_months_ago = (datetime.today() - timedelta(days=90)).replace(tzinfo=pytz.UTC).strftime('%Y-%m-%d')
        six_months_ago = (datetime.today() - timedelta(days=180)).replace(tzinfo=pytz.UTC).strftime('%Y-%m-%d')
        twelve_months_ago = (datetime.today() - timedelta(days=365)).replace(tzinfo=pytz.UTC).strftime('%Y-%m-%d')

        current_price = stock_data.iloc[-1] if not stock_data.empty else None
        print("CurrentPrice", current_price)
        three_months_price = get_nearest_price(stock_data, three_months_ago)
        six_months_price = get_nearest_price(stock_data, six_months_ago)
        twelve_months_price = get_nearest_price(stock_data, twelve_months_ago)

        three_months_return = ((current_price - three_months_price) / three_months_price * 100) if three_months_price else None
        six_months_return = ((current_price - six_months_price) / six_months_price * 100) if six_months_price else None
        twelve_months_return = ((current_price - twelve_months_price) / twelve_months_price * 100) if twelve_months_price else None

        return [ticker, three_months_return, six_months_return, twelve_months_return]
    except Exception as e:
        print(f"Error calculating returns for {ticker}: {e}")
        return [ticker, None, None, None]

# Read stock tickers from CSV
input_file = "tickers.csv"  # Ensure this file exists with a column 'Ticker'
output_file = "stock_returns.xlsx"

df = pd.read_csv(input_file)
print(df)
tickers = df['Ticker'].tolist()

# Fetch stock data for all tickers and calculate returns
results = []
for ticker in tickers:
    stock_data = fetch_stock_data(ticker)
    results.append(calculate_returns(ticker, stock_data))

# Save results to Excel
output_df = pd.DataFrame(results, columns=["Ticker", "3M Return (%)", "6M Return (%)", "12M Return (%)"])
output_df.to_excel(output_file, index=False)

print(f"Stock returns saved to {output_file}")


         Ticker
0        360ONE
1       3MINDIA
2           ABB
3           ACC
4        AIAENG
..          ...
495        ZEEL
496  ZENSARTECH
497      ZOMATO
498   ZYDUSLIFE
499      ECLERX

[500 rows x 1 columns]
Fetching data for 360ONE.NS with daily interval for 1 year
Date
2024-03-20 18:30:00+00:00    668.708130
2024-03-21 18:30:00+00:00    661.210754
2024-03-25 18:30:00+00:00    645.769104
2024-03-26 18:30:00+00:00    663.296143
2024-03-27 18:30:00+00:00    670.892822
                                ...    
2025-03-16 18:30:00+00:00    854.599976
2025-03-17 18:30:00+00:00    854.599976
2025-03-18 18:30:00+00:00    923.799988
2025-03-19 18:30:00+00:00    926.599976
2025-03-20 18:30:00+00:00    930.000000
Name: Close, Length: 248, dtype: float64
CurrentPrice 930.0
Fetching data for 3MINDIA.NS with daily interval for 1 year
Date
2024-03-20 18:30:00+00:00    29650.115234
2024-03-21 18:30:00+00:00    30320.246094
2024-03-25 18:30:00+00:00    30082.199219
2024-03-26 18:30:00+00:00    

$HEG.NS: possibly delisted; no price data found  (period=1y)


No data available for HEG.NS
None
Skipping return calculation for HEG due to missing data.
Fetching data for HBLENGINE.NS with daily interval for 1 year
Date
2024-12-11 18:30:00+00:00    670.799988
2024-12-12 18:30:00+00:00    695.000000
2024-12-15 18:30:00+00:00    704.900024
2024-12-16 18:30:00+00:00    701.599976
2024-12-17 18:30:00+00:00    679.299988
                                ...    
2025-03-16 18:30:00+00:00    406.399994
2025-03-17 18:30:00+00:00    418.750000
2025-03-18 18:30:00+00:00    482.399994
2025-03-19 18:30:00+00:00    460.100006
2025-03-20 18:30:00+00:00    466.000000
Name: Close, Length: 70, dtype: float64
CurrentPrice 466.0
Fetching data for HCLTECH.NS with daily interval for 1 year
Date
2024-03-20 18:30:00+00:00    1542.112915
2024-03-21 18:30:00+00:00    1505.439575
2024-03-25 18:30:00+00:00    1509.691528
2024-03-26 18:30:00+00:00    1494.181396
2024-03-27 18:30:00+00:00    1491.620605
                                ...     
2025-03-16 18:30:00+00:00    154

$RITES.NS: possibly delisted; no price data found  (period=1y)


No data available for RITES.NS
None
Skipping return calculation for RITES due to missing data.
Fetching data for RADICO.NS with daily interval for 1 year
Date
2024-03-20 18:30:00+00:00    1607.839844
2024-03-21 18:30:00+00:00    1640.432861
2024-03-25 18:30:00+00:00    1671.079346
2024-03-26 18:30:00+00:00    1667.535400
2024-03-27 18:30:00+00:00    1724.735474
                                ...     
2025-03-16 18:30:00+00:00    2214.350098
2025-03-17 18:30:00+00:00    2214.350098
2025-03-18 18:30:00+00:00    2272.300049
2025-03-19 18:30:00+00:00    2286.449951
2025-03-20 18:30:00+00:00    2264.000000
Name: Close, Length: 248, dtype: float64
CurrentPrice 2264.0
Fetching data for RVNL.NS with daily interval for 1 year
Date
2024-03-20 18:30:00+00:00    243.552094
2024-03-21 18:30:00+00:00    248.881363
2024-03-25 18:30:00+00:00    259.440277
2024-03-26 18:30:00+00:00    255.007507
2024-03-27 18:30:00+00:00    251.919525
                                ...    
2025-03-16 18:30:00+00:00  